<a href="https://colab.research.google.com/github/1kaiser/GRAINet/blob/main/Depth_Grading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ViT tiny depth and grade map generation

### 📦 SECTION 1: IMPORTS AND ENVIRONMENT SETUP
#

In [1]:
print("🚀 Setting up environment...")
import os
# os.environ['JAX_PLATFORM_NAME'] = 'cpu'

# Core imports
import sys
import numpy as np
import jax
import jax.numpy as jnp
from flax import nnx
from PIL import Image
from tqdm import tqdm
import urllib.request
from typing import Dict, List, Tuple
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pickle
import os
from pathlib import Path
from datetime import datetime

# JAX configuration
jax.config.update('jax_enable_x64', False)

print("✅ Environment setup complete!")
print(f"🔧 JAX version: {jax.__version__}")
print(f"🔧 NumPy version: {np.__version__}")

🚀 Setting up environment...
✅ Environment setup complete!
🔧 JAX version: 0.5.3
🔧 NumPy version: 2.0.2


### 🔗 SECTION 2: DOWNLOAD ORIGINAL STRING ENCODER
#

In [2]:

print("\n" + "="*70)
print("🔗 SECTION 2: NNX STRING CAYLEY ATTENTION IMPLEMENTATION")
print("="*70)

class StringPositionEmbedding2D(nnx.Module):
    """
    STRING CAYLEY: Separable Translationally Invariant Position Encodings.
    Simplified NNX implementation using only Cayley transform.
    """

    def __init__(self, seq_len: int, embed_dim: int, *, rngs: nnx.Rngs):
        self.seq_len = seq_len
        self.embed_dim = embed_dim  # This is the dimension for a single axis (e.g., HE/2)

        # Cayley-STRING: Initialize a learnable matrix for the generator
        self.S = nnx.Param(
            nnx.initializers.normal(stddev=0.01)(rngs.params(), (self.embed_dim, self.embed_dim))
        )

        # --- Non-Learnable Buffers (Constants) ---
        # Positional indices (0 for CLS, 1 to N for patches)
        self.positions = jnp.arange(self.seq_len, dtype=jnp.float32)

        # Pre-compute base RoPE frequencies for half the dimensions
        self.freqs = 1.0 / (10000 ** (jnp.arange(0, self.embed_dim // 2, dtype=jnp.float32) * 2 / self.embed_dim))

    def _apply_efficient_rope(self, x):
        """Applies RoPE rotation directly to vectors without creating a large matrix."""
        # x shape: (B, H, S, E)
        # Calculate sin/cos factors for each position
        angles = jnp.outer(self.positions, self.freqs)  # (S, E/2)
        cos_vals = jnp.cos(angles)  # (S, E/2)
        sin_vals = jnp.sin(angles)  # (S, E/2)

        # Repeat to match the full embedding dimension
        cos_vals = jnp.repeat(cos_vals, 2, axis=-1)  # (S, E)
        sin_vals = jnp.repeat(sin_vals, 2, axis=-1)  # (S, E)

        # Apply the 2D rotation formula: x_rot = x*cos - permute(x)*sin
        x1, x2 = jnp.split(x, 2, axis=-1)
        x_permuted = jnp.concatenate([-x2, x1], axis=-1)

        # Broadcast across batch and head dimensions
        x_rotated = x * cos_vals[None, None, :, :] + x_permuted * sin_vals[None, None, :, :]
        return x_rotated

    def __call__(self, x):
        """Applies STRING CAYLEY positional encoding."""
        # 1. Generate the learnable orthogonal transformation matrix P using Cayley transform
        # Make S antisymmetric: (S - S^T)/2
        S_antisym = (self.S.value - self.S.value.T) / 2.0
        # Cayley Transform using linear solver for stability, as recommended
        I = jnp.eye(self.embed_dim, dtype=x.dtype)
        P = jnp.linalg.solve(I + S_antisym, I - S_antisym)

        # 2. Apply the learnable transformation P to the input
        # (B, H, S, E) @ (E, E) -> (B, H, S, E)
        x_transformed = jnp.matmul(x, P.T)

        # 3. Apply RoPE rotation efficiently to the transformed input
        return self._apply_efficient_rope(x_transformed)


class SelfAttentionWithStringCayley(nnx.Module):
    """Self-attention module with STRING CAYLEY positional encoding (NNX version)."""

    def __init__(self, embed_dim: int, n_attention_heads: int, seq_len: int, *, rngs: nnx.Rngs):
        self.embed_dim = embed_dim
        self.n_attention_heads = n_attention_heads
        self.seq_len = seq_len

        self.head_embed_dim = self.embed_dim // self.n_attention_heads
        if self.head_embed_dim * self.n_attention_heads != self.embed_dim:
            raise ValueError("embed_dim must be divisible by n_attention_heads")

        # Linear projections for Q, K, V (NNX uses nnx.Linear instead of nn.Dense)
        self.queries = nnx.Linear(
            in_features=embed_dim,
            out_features=embed_dim,
            use_bias=False,
            rngs=rngs
        )
        self.keys = nnx.Linear(
            in_features=embed_dim,
            out_features=embed_dim,
            use_bias=False,
            rngs=rngs
        )
        self.values = nnx.Linear(
            in_features=embed_dim,
            out_features=embed_dim,
            use_bias=False,
            rngs=rngs
        )
        self.out_projection = nnx.Linear(
            in_features=embed_dim,
            out_features=embed_dim,
            rngs=rngs
        )

        # Split head_embed_dim for x and y axes, handling odd dimensions
        self.dim_split_x = self.head_embed_dim // 2 + (self.head_embed_dim % 2)
        self.dim_split_y = self.head_embed_dim // 2

        # STRING CAYLEY positional encodings for x and y axes
        self.string_x = StringPositionEmbedding2D(
            seq_len=self.seq_len,
            embed_dim=self.dim_split_x,
            rngs=rngs
        )
        self.string_y = StringPositionEmbedding2D(
            seq_len=self.seq_len,
            embed_dim=self.dim_split_y,
            rngs=rngs
        )

    def __call__(self, x):
        b, s, e = x.shape

        # Generate Q, K, V and reshape to (B, H, S, HE)
        xq = self.queries(x).reshape(b, s, self.n_attention_heads, self.head_embed_dim)
        xq = jnp.transpose(xq, (0, 2, 1, 3))
        xk = self.keys(x).reshape(b, s, self.n_attention_heads, self.head_embed_dim)
        xk = jnp.transpose(xk, (0, 2, 1, 3))
        xv = self.values(x).reshape(b, s, self.n_attention_heads, self.head_embed_dim)
        xv = jnp.transpose(xv, (0, 2, 1, 3))

        # Split queries and keys for x and y axes
        xq_x, xq_y = jnp.split(xq, [self.dim_split_x], axis=-1)
        xk_x, xk_y = jnp.split(xk, [self.dim_split_x], axis=-1)

        # Apply STRING positional encoding to each axis
        xq_x_str = self.string_x(xq_x)
        xq_y_str = self.string_y(xq_y)
        xk_x_str = self.string_x(xk_x)
        xk_y_str = self.string_y(xk_y)

        # Concatenate the results
        xq = jnp.concatenate([xq_x_str, xq_y_str], axis=-1)
        xk = jnp.concatenate([xk_x_str, xk_y_str], axis=-1)

        # Standard attention computation
        xk_t = jnp.transpose(xk, (0, 1, 3, 2))
        x_attention = jnp.matmul(xq, xk_t) / np.sqrt(self.head_embed_dim)
        x_attention = jax.nn.softmax(x_attention, axis=-1)

        x = jnp.matmul(x_attention, xv)

        # Reshape and project output
        x = jnp.transpose(x, (0, 2, 1, 3))
        x = x.reshape(b, s, e)
        x = self.out_projection(x)

        return x

print("✅ NNX STRING CAYLEY attention mechanism implemented!")



🔗 SECTION 2: NNX STRING CAYLEY ATTENTION IMPLEMENTATION
✅ NNX STRING CAYLEY attention mechanism implemented!


### 🏗️ SECTION 3: MODEL ARCHITECTURE


In [3]:

print("\n" + "="*70)
print("🏗️ SECTION 3: COMPLETE MODEL ARCHITECTURE (NNX)")
print("="*70)

class TinyViTBlockStringCayley(nnx.Module):
    """
    🧱 Tiny ViT block using NNX STRING CAYLEY attention
    ✨ Features: STRING CAYLEY attention + MLP with residual connections
    """

    def __init__(self, embed_dim: int, n_heads: int, seq_len: int, mlp_ratio: int = 4, *, rngs: nnx.Rngs):
        self.embed_dim = embed_dim
        self.n_heads = n_heads
        self.seq_len = seq_len
        self.mlp_ratio = mlp_ratio

        # Layer normalization
        self.norm1 = nnx.LayerNorm(embed_dim, rngs=rngs)
        self.norm2 = nnx.LayerNorm(embed_dim, rngs=rngs)

        # NNX STRING CAYLEY attention
        self.string_attention = SelfAttentionWithStringCayley(
            embed_dim=embed_dim,
            n_attention_heads=n_heads,
            seq_len=seq_len,
            rngs=rngs
        )

        # MLP block
        mlp_dim = embed_dim * mlp_ratio
        self.mlp_dense1 = nnx.Linear(embed_dim, mlp_dim, rngs=rngs)
        self.mlp_dense2 = nnx.Linear(mlp_dim, embed_dim, rngs=rngs)

    def __call__(self, x, training: bool = True, return_features: bool = False):
        # STRING attention with residual
        normed_x = self.norm1(x)
        attn_output = self.string_attention(normed_x)
        x_after_attn = x + attn_output

        # MLP with residual
        mlp_out = self.mlp_dense1(self.norm2(x_after_attn))
        mlp_out = nnx.gelu(mlp_out)
        mlp_out = self.mlp_dense2(mlp_out)
        x_final = x_after_attn + mlp_out

        if return_features:
            return x_final, x_final
        return x_final

class DirectStringViTComplete(nnx.Module):
    """
    🎯 Complete ViT with NNX STRING CAYLEY encoder for dual spatial prediction
    📊 Outputs: Depth maps + Grading maps (14x14 each)
    """

    def __init__(self, patch_size: int = 16, embed_dim: int = 64, n_heads: int = 4, n_layers: int = 4, *, rngs: nnx.Rngs):
        self.patch_size = patch_size
        self.embed_dim = embed_dim
        self.n_heads = n_heads
        self.n_layers = n_layers

        # Patch embedding layer
        self.patch_embed = nnx.Conv(
            in_features=3,
            out_features=embed_dim,
            kernel_size=(patch_size, patch_size),
            strides=(patch_size, patch_size),
            rngs=rngs
        )

        # Learnable CLS token
        self.cls_token = nnx.Param(nnx.initializers.normal(stddev=0.02)(rngs.params(), (1, 1, embed_dim)))

        # Calculate sequence length: 224x224 -> 14x14 patches + 1 CLS
        seq_len = (224 // patch_size) ** 2 + 1  # 196 + 1 = 197

        # Stack of ViT blocks with STRING CAYLEY encoders
        self.blocks = [
            TinyViTBlockStringCayley(
                embed_dim=embed_dim,
                n_heads=n_heads,
                seq_len=seq_len,
                rngs=rngs
            ) for _ in range(n_layers)
        ]

        # Final layer normalization
        self.norm = nnx.LayerNorm(embed_dim, rngs=rngs)

        # Dual prediction heads
        self.depth_head = nnx.Linear(embed_dim, 1, rngs=rngs)
        self.grading_head = nnx.Linear(embed_dim, 1, rngs=rngs)

    def __call__(self, x, training: bool = True, return_attention: bool = False):
        B = x.shape[0]

        # Patch embedding: 224x224x3 -> 14x14x64 -> 196x64
        patches = self.patch_embed(x)
        patches = patches.reshape(B, -1, self.embed_dim)

        # Add CLS token: 196 + 1 = 197 tokens
        cls_tokens = jnp.broadcast_to(self.cls_token.value, (B, 1, self.embed_dim))
        x = jnp.concatenate([cls_tokens, patches], axis=1)

        # Apply ViT blocks and collect feature maps from each layer
        layer_features = []
        if return_attention:
            print(f"🔍 Starting layer processing with input shape: {x.shape}")

        for i, block in enumerate(self.blocks):
            if return_attention:
                x, layer_output = block(x, training=training, return_features=True)
                if return_attention:
                    print(f"   Layer {i+1}: Block output shape: {layer_output.shape}")
                # Store patch tokens (exclude CLS) for feature map visualization
                patch_features = layer_output[:, 1:]  # B x patches x embed_dim
                if return_attention:
                    print(f"   Layer {i+1}: Patch features shape: {patch_features.shape}")
                layer_features.append(patch_features)
            else:
                x = block(x, training=training)

        x = self.norm(x)

        # Extract patch tokens (exclude CLS)
        patch_tokens = x[:, 1:]  # B x 196 x 64

        # Dual spatial predictions
        depth_features = self.depth_head(patch_tokens).squeeze(-1)
        grading_features = self.grading_head(patch_tokens).squeeze(-1)

        # Reshape to spatial maps (14x14)
        depth_map = depth_features.reshape(B, 14, 14)
        grading_map = grading_features.reshape(B, 14, 14)

        if return_attention:
            return depth_map, grading_map, layer_features
        return depth_map, grading_map

print("✅ Complete model architecture defined successfully!")
print("🏗️ Components:")
print("   📦 TinyViTBlockStringCayley - STRING CAYLEY attention + MLP")
print("   🎯 DirectStringViTComplete - Complete model with dual outputs")



🏗️ SECTION 3: COMPLETE MODEL ARCHITECTURE (NNX)
✅ Complete model architecture defined successfully!
🏗️ Components:
   📦 TinyViTBlockStringCayley - STRING CAYLEY attention + MLP
   🎯 DirectStringViTComplete - Complete model with dual outputs


### 📊 SECTION 4: DATA LOADING AND PREPROCESSING
#

In [4]:
!wget -nc -q https://github.com/1kaiser/GRAINet/releases/download/1/input_depth.zip https://github.com/1kaiser/GRAINet/releases/download/1/GRAINet_demo_data.zip

!unzip -o /content/input_depth.zip > /dev/null 2>&1
!unzip -o /content/GRAINet_demo_data.zip > /dev/null 2>&1

In [19]:

print("\n" + "="*70)
print("📊 SECTION 4: DATA LOADING")
print("="*70)

def create_spatial_grading_target(grain_size, spatial_size=14):
    """
    🎯 Create realistic spatial grading map with variation
    📏 Size: 14x14 to match model output
    """
    base_map = np.full((spatial_size, spatial_size), grain_size, dtype=np.float32)
    variation = np.random.normal(0, grain_size * 0.1, (spatial_size, spatial_size))
    grading_map = base_map + variation
    return np.maximum(grading_map, 0.1)  # Ensure positive values

def load_dataset_for_training(input_dir='/content', n_samples=20):
    """
    📂 Load training dataset with images, depth maps, and grain labels
    📄 Preprocessing: Resize, normalize, create targets
    """
    print(f"📂 Loading dataset from: {input_dir}")
    print(f"📊 Number of samples: {n_samples}")

    # Try to load grain size labels
    grain_data_path = 'data_GRAINet_demo/data_KLEmme_1bank.npz'

    data = np.load(grain_data_path)
    grain_labels = data['dm']
    print(f"✅ Grain labels loaded: {len(grain_labels)} samples")


    images = []
    depth_maps = []
    grading_maps = []

    print("📄 Processing samples...")
    for i in tqdm(range(n_samples), desc="Loading"):
        # Load input image
        img_path = f'{input_dir}/inputimage/grain_{i:04d}.jpg'
        img = Image.open(img_path).resize((224, 224))
        images.append(np.array(img).astype(np.float32) / 255.0)


        # Load depth map
        depth_path = f'{input_dir}/depthimage/depth_{i:04d}.png'

        depth = Image.open(depth_path).resize((14, 14), Image.LANCZOS)
        depth_array = np.array(depth).astype(np.float32) / 255.0
        if len(depth_array.shape) == 3:
            depth_array = depth_array[:, :, 0]  # Take first channel
        depth_maps.append(depth_array)


        # Create grading map
        grain_size = grain_labels[i] if i < len(grain_labels) else np.random.uniform(3.0, 8.0)
        grading_map = create_spatial_grading_target(grain_size)
        grading_maps.append(grading_map)

    print(f"✅ Dataset loaded successfully!")
    print(f"📊 Final shapes:")
    print(f"   🖼️ Images: {np.array(images).shape}")
    print(f"   🏔️ Depth maps: {np.array(depth_maps).shape}")
    print(f"   📏 Grading maps: {np.array(grading_maps).shape}")

    return np.array(images), np.array(depth_maps), np.array(grading_maps)



📊 SECTION 4: DATA LOADING


### 📊 SECTION 5: CHECKPOINTING

In [6]:

print("\n" + "="*70)
print("💾 SECTION 5: SIMPLE PICKLE CHECKPOINTING")
print("="*70)

# Simple checkpoint directory
checkpoint_dir = Path('simple_checkpoints')
checkpoint_dir.mkdir(exist_ok=True)

def save_checkpoint(model, model_config: Dict, epoch: int, loss_history: List[Dict]):
    """💾 Save checkpoint using simple pickle (JAX/Orbax compatibility fix)"""
    try:
        data = {
            'model_state': nnx.state(model),
            'config': model_config,
            'epoch': epoch,
            'history': loss_history
        }
        checkpoint_path = checkpoint_dir / f'checkpoint_epoch_{epoch}.pkl'
        with open(checkpoint_path, 'wb') as f:
            pickle.dump(data, f)
        print(f"   💾 Checkpoint saved: {checkpoint_path}")
        return epoch
    except Exception as e:
        print(f"   ⚠️ Checkpoint save failed: {e}")
        return epoch

def load_checkpoint(epoch: int = None):
    """📂 Load checkpoint using simple pickle"""
    try:
        if epoch is None:
            # Find latest checkpoint
            checkpoints = list(checkpoint_dir.glob('checkpoint_epoch_*.pkl'))
            if not checkpoints:
                raise FileNotFoundError("No checkpoints found")
            latest = max(checkpoints, key=lambda p: int(p.stem.split('_')[-1]))
            epoch = int(latest.stem.split('_')[-1])

        checkpoint_path = checkpoint_dir / f'checkpoint_epoch_{epoch}.pkl'
        with open(checkpoint_path, 'rb') as f:
            data = pickle.load(f)

        print(f"✅ Loaded checkpoint epoch {epoch}")
        return data['model_state'], data['config'], data['epoch'], data['history']
    except Exception as e:
        print(f"❌ Checkpoint load failed: {e}")
        return None, None, None, None

def list_checkpoints():
    """📋 List available checkpoints"""
    try:
        checkpoints = list(checkpoint_dir.glob('checkpoint_epoch_*.pkl'))
        epochs = [int(p.stem.split('_')[-1]) for p in checkpoints]
        if epochs:
            print(f"📋 Available epochs: {sorted(epochs)}")
        else:
            print("📋 No checkpoints found")
        return sorted(epochs)
    except Exception as e:
        print(f"❌ Error listing checkpoints: {e}")
        return []

print("✅ Simple pickle checkpointing ready (JAX/Orbax compatibility fix)!")



💾 SECTION 5: SIMPLE PICKLE CHECKPOINTING
✅ Simple pickle checkpointing ready (JAX/Orbax compatibility fix)!


### 🚀 SECTION 6: TRAINING FUNCTIONS

In [20]:

print("\n" + "="*70)
print("🚀 SECTION 6: TRAINING SETUP")
print("="*70)

def simple_sgd_update(model, grads, learning_rate=0.001):
    """📉 Simple SGD parameter update with learning rate for NNX models"""
    try:
        nnx.update(model, jax.tree.map(lambda p, g: p - learning_rate * g, nnx.state(model), grads))
    except AttributeError:
        nnx.update(model, jax.tree_util.tree_map(lambda p, g: p - learning_rate * g, nnx.state(model), grads))

def create_dual_loss_function():
    """
    🎯 Create loss function for dual spatial outputs
    📊 Combines depth and grading losses
    """
    def loss_fn(model, batch_x, batch_depth, batch_grading):
        depth_pred, grading_pred = model(batch_x, training=True)

        # Individual losses
        depth_loss = jnp.mean((depth_pred - batch_depth) ** 2)
        grading_loss = jnp.mean((grading_pred - batch_grading) ** 2)

        # Combined loss
        total_loss = 0.9 *depth_loss + 0.1 * grading_loss

        return total_loss, {
            'depth_loss': depth_loss,
            'grading_loss': grading_loss,
            'total_loss': total_loss
        }

    return loss_fn

def train_complete_string_vit(input_dir='/content', n_samples=50, epochs=10, batch_size=8,
                             learning_rate=0.001, n_layers=4, embed_dim=64, n_heads=4,
                             patch_size=16, save_checkpoints=True, checkpoint_interval=5):
    """
    🏃‍♂️ Main training function with complete NNX STRING CAYLEY implementation
    🎯 Features: Real data support, comprehensive training, progress tracking
    """
    print("\n" + "="*80)
    print("🚀 STARTING COMPLETE STRING CAYLEY VIT TRAINING")
    print("="*80)
    print("✅ Using NNX-compatible STRING CAYLEY attention!")
    print("📊 Real-time training with dual spatial outputs!")
    print("="*80)

    # Try to load real dataset first, fall back to synthetic
    print("\n📂 Loading training dataset...")
    images, depth_targets, grading_targets = load_dataset_for_training(
        input_dir=input_dir, n_samples=n_samples
    )
    print("✅ Real dataset loaded successfully!")


    # Convert to JAX arrays
    X = jnp.array(images)
    depth_targets = jnp.array(depth_targets)
    grading_targets = jnp.array(grading_targets)

    print(f"\n📊 DATASET SUMMARY:")
    print(f"   🖼️ Images: {X.shape}")
    print(f"   🏔️ Depth targets: {depth_targets.shape}")
    print(f"   📏 Grading targets: {grading_targets.shape}")

    # Initialize model
    print(f"\n🏗️ Initializing DirectStringViTComplete...")
    print(f"⚙️ Model Configuration:")
    print(f"   🔢 Layers: {n_layers}")
    print(f"   📐 Embed Dim: {embed_dim}")
    print(f"   🧠 Attention Heads: {n_heads}")
    print(f"   📦 Patch Size: {patch_size}x{patch_size}")
    print(f"   💾 Checkpointing: {'Enabled' if save_checkpoints else 'Disabled'}")
    if save_checkpoints:
        print(f"   📅 Save Interval: Every {checkpoint_interval} epochs")

    # Store model configuration
    model_config = {
        'n_layers': n_layers,
        'embed_dim': embed_dim,
        'n_heads': n_heads,
        'patch_size': patch_size,
        'learning_rate': learning_rate,
        'batch_size': batch_size
    }

    # Initialize NNX model
    rngs = nnx.Rngs(42)
    model = DirectStringViTComplete(
        patch_size=patch_size,
        embed_dim=embed_dim,
        n_heads=n_heads,
        n_layers=n_layers,
        rngs=rngs
    )
    print("✅ NNX Model initialized with STRING CAYLEY encoder!")

    # Test forward pass
    print(f"\n🔄 Testing forward pass...")
    depth_pred, grading_pred = model(X[:2], training=False)
    print(f"✅ Forward pass successful!")
    print(f"   🏔️ Depth output: {depth_pred.shape}")
    print(f"   📏 Grading output: {grading_pred.shape}")

    # Setup training components
    loss_fn = create_dual_loss_function()

    def train_step(model, batch_x, batch_depth, batch_grading):
        def compute_loss(model_state):
            temp_model = nnx.clone(model)
            nnx.update(temp_model, model_state)
            return loss_fn(temp_model, batch_x, batch_depth, batch_grading)

        (loss, metrics), grads = nnx.value_and_grad(compute_loss, has_aux=True)(nnx.state(model))
        simple_sgd_update(model, grads, learning_rate=learning_rate)
        return metrics

    # Training loop
    print(f"\n🏃‍♂️ TRAINING LOOP START")
    print(f"⚙️ Configuration:")
    print(f"   📊 Epochs: {epochs}")
    print(f"   📄 Batch size: {batch_size}")
    print(f"   📉 Learning rate: {learning_rate}")
    print(f"   🎯 Samples: {n_samples}")
    print("="*50)

    loss_history = []

    for epoch in range(epochs):
        # Training step
        batch_x = X[:batch_size]
        batch_depth = depth_targets[:batch_size]
        batch_grading = grading_targets[:batch_size]

        metrics = train_step(model, batch_x, batch_depth, batch_grading)

        # Store metrics
        loss_history.append({
            'depth_loss': float(metrics['depth_loss']),
            'grading_loss': float(metrics['grading_loss']),
            'total_loss': float(metrics['total_loss'])
        })

        # Print epoch results
        print(f"🏃‍♂️ Epoch {epoch+1:2d}: "
              f"Total={metrics['total_loss']:.4f} | "
              f"Depth={metrics['depth_loss']:.4f} | "
              f"Grading={metrics['grading_loss']:.4f}")

        # Test inference with attention maps every 5 epochs
        if (epoch + 1) % 5 == 0:
            print(f"   🔍 Testing attention maps...")
            try:
                depth_pred, grading_pred, layer_features = model(X[:1], training=False, return_attention=True)
                print(f"   ✅ Attention extraction successful!")
                print(f"       🧠 Layer features: {len(layer_features)} layers")
                if layer_features:
                    print(f"       📊 First layer shape: {layer_features[0].shape}")
            except Exception as e:
                print(f"   ⚠️ Attention extraction failed: {e}")

        # Save checkpoint at specified intervals
        if save_checkpoints and (epoch + 1) % checkpoint_interval == 0:
            checkpoint_path = save_checkpoint(
                model=model,
                model_config=model_config,
                epoch=epoch + 1,
                loss_history=loss_history
            )
            print(f"   💾 Checkpoint saved: epoch {epoch + 1}")

            # IMMEDIATE INFERENCE & VISUALIZATION AFTER CHECKPOINT
            print(f"   📊 Creating comprehensive visualization for epoch {epoch + 1}...")
            try:
                # Prepare sample data for visualization
                sample_data = (X[:1], depth_targets[:1], grading_targets[:1])

                if (epoch + 1) % 10 == 0:
                  from google.colab import output
                  output.clear()

                # Create comprehensive plot
                save_path = f'training_inference_epoch_{epoch + 1:03d}.png'
                depth_pred, grading_pred, layer_features = comprehensive_inference_plot(
                    model=model,
                    test_data=sample_data,
                    loss_history=loss_history,
                    epoch=epoch + 1,
                    save_path=save_path
                )

                print(f"   ✅ Comprehensive plot generated: {save_path}")
                print(f"   📊 Depth prediction range: [{float(jnp.min(depth_pred)):.4f}, {float(jnp.max(depth_pred)):.4f}]")
                print(f"   📊 Grading prediction range: [{float(jnp.min(grading_pred)):.2f}, {float(jnp.max(grading_pred)):.2f}]")

            except Exception as viz_error:
                print(f"   ⚠️ Visualization failed: {viz_error}")
                # Continue training even if visualization fails

    # Final analysis
    print(f"\n🎉 TRAINING COMPLETED!")
    print("="*50)
    print("🏆 FINAL RESULTS:")

    final_metrics = loss_history[-1]
    print(f"   🎯 Final Total Loss: {final_metrics['total_loss']:.6f}")
    print(f"   🏔️ Final Depth Loss: {final_metrics['depth_loss']:.6f}")
    print(f"   📏 Final Grading Loss: {final_metrics['grading_loss']:.6f}")

    improvement = (loss_history[0]['total_loss'] - final_metrics['total_loss']) / loss_history[0]['total_loss'] * 100
    print(f"   📈 Total Improvement: {improvement:.1f}%")

    print(f"\n✨ KEY ACHIEVEMENTS:")
    print(f"   ✅ STRING CAYLEY encoder working successfully")
    print(f"   ✅ Dual spatial outputs (depth + grading) functional")
    print(f"   ✅ Training convergence achieved")
    print(f"   ✅ No compatibility errors")
    print(f"   ✅ Ready for production use")

    # Test final inference
    print(f"\n🔮 FINAL INFERENCE TEST:")
    try:
        test_depth, test_grading, test_features = model(X[:1], training=False, return_attention=True)
        print(f"✅ Final inference successful!")
        print(f"   🏔️ Depth prediction shape: {test_depth.shape}")
        print(f"   📏 Grading prediction shape: {test_grading.shape}")
        print(f"   🧠 Feature maps from {len(test_features)} layers")

        # Show some prediction statistics
        print(f"   📊 Depth prediction stats:")
        print(f"       Mean: {float(jnp.mean(test_depth)):.4f}")
        print(f"       Std: {float(jnp.std(test_depth)):.4f}")
        print(f"   📊 Grading prediction stats:")
        print(f"       Mean: {float(jnp.mean(test_grading)):.4f}")
        print(f"       Std: {float(jnp.std(test_grading)):.4f}")

    except Exception as e:
        print(f"❌ Final inference failed: {e}")

    # Save final checkpoint
    if save_checkpoints:
        final_checkpoint = save_checkpoint(
            model=model,
            model_config=model_config,
            epoch=epochs,
            loss_history=loss_history
        )
        print(f"💾 Final checkpoint saved")
        print(f"📋 All saved checkpoints:")
        list_checkpoints()

    return model, loss_history, model_config

print("✅ Training functions ready!")



🚀 SECTION 6: TRAINING SETUP
✅ Training functions ready!


### 🚀 SECTION 6: 6.5 VISUALIZATION CODE

In [8]:
# @title ### 🚀 SECTION 6: 6.5 VISUALIZATION CODE
print("\n" + "="*70)
print("📊 SECTION 6.5: COMPREHENSIVE VISUALIZATION")
print("="*70)

def load_model_from_checkpoint(epoch: int = None):
    """🔄 Load complete model from checkpoint"""
    model_state, config, epoch_loaded, history = load_checkpoint(epoch)
    if model_state is None:
        return None, None, None, None

    # Recreate model with same configuration
    rngs = nnx.Rngs(42)
    model = DirectStringViTComplete(
        patch_size=config['patch_size'],
        embed_dim=config['embed_dim'],
        n_heads=config['n_heads'],
        n_layers=config['n_layers'],
        rngs=rngs
    )

    # Restore model state
    nnx.update(model, model_state)

    print(f"✅ Model loaded from checkpoint epoch {epoch_loaded}")
    return model, config, epoch_loaded, history

def comprehensive_inference_plot(model, test_data, loss_history, epoch, save_path=None):
    """
    📊 Create comprehensive visualization with all components
    Shows: Loss curves, input, targets, predictions, and all feature maps
    """
    test_x, test_depth, test_grading = test_data

    # Run inference with attention extraction
    depth_pred, grading_pred, layer_features = model(test_x[:1], training=False, return_attention=True)

    # Convert to numpy for plotting
    sample_img = np.array(test_x[0])
    sample_depth_target = np.array(test_depth[0])
    sample_grading_target = np.array(test_grading[0])
    depth_prediction = np.array(depth_pred[0])
    grading_prediction = np.array(grading_pred[0])

    # Calculate figure layout
    n_layers = len(layer_features)

    # Create figure with comprehensive layout
    fig = plt.figure(figsize=(24, 16))

    # Create grid: 4 rows x 6 columns
    gs = gridspec.GridSpec(4, 6, figure=fig, hspace=0.3, wspace=0.3)

    # === ROW 1: Main Results ===
    ax_loss = fig.add_subplot(gs[0, :2])
    ax_input = fig.add_subplot(gs[0, 2])
    ax_target_depth = fig.add_subplot(gs[0, 3])
    ax_pred_depth = fig.add_subplot(gs[0, 4])
    ax_metrics = fig.add_subplot(gs[0, 5])

    # === ROW 2: Grading Results ===
    ax_target_grading = fig.add_subplot(gs[1, 0])
    ax_pred_grading = fig.add_subplot(gs[1, 1])
    ax_depth_diff = fig.add_subplot(gs[1, 2])
    ax_grading_diff = fig.add_subplot(gs[1, 3])
    ax_combined = fig.add_subplot(gs[1, 4])
    ax_stats = fig.add_subplot(gs[1, 5])

    # === ROWS 3-4: Feature Maps ===
    feature_axes = []
    for row in [2, 3]:
        for col in range(6):
            feature_axes.append(fig.add_subplot(gs[row, col]))

    # 1. Loss Curves
    if loss_history:
        epochs = list(range(1, len(loss_history) + 1))
        total_losses = [h['total_loss'] for h in loss_history]
        depth_losses = [h['depth_loss'] for h in loss_history]
        grading_losses = [h['grading_loss'] for h in loss_history]

        ax_loss.semilogy(epochs, total_losses, 'b-', linewidth=3, label='Total Loss', marker='o')
        ax_loss.semilogy(epochs, depth_losses, 'r--', linewidth=2, label='Depth Loss', marker='s')
        ax_loss.semilogy(epochs, grading_losses, 'g--', linewidth=2, label='Grading Loss', marker='^')
        ax_loss.set_xlabel('Epoch')
        ax_loss.set_ylabel('Loss (log scale)')
        ax_loss.set_title(f'Training Progress (Epoch {epoch})', fontweight='bold')
        ax_loss.legend()
        ax_loss.grid(True, alpha=0.3)

    # 2. Input Image
    ax_input.imshow(sample_img)
    ax_input.set_title('Input Image', fontweight='bold')
    ax_input.axis('off')

    # 3. Target Depth
    im_depth_target = ax_target_depth.imshow(sample_depth_target, cmap='viridis')
    ax_target_depth.set_title('Target Depth', fontweight='bold')
    ax_target_depth.axis('off')
    plt.colorbar(im_depth_target, ax=ax_target_depth, fraction=0.046, pad=0.04)

    # 4. Predicted Depth
    im_depth_pred = ax_pred_depth.imshow(depth_prediction, cmap='viridis')
    ax_pred_depth.set_title('Predicted Depth', fontweight='bold')
    ax_pred_depth.axis('off')
    plt.colorbar(im_depth_pred, ax=ax_pred_depth, fraction=0.046, pad=0.04)

    # 5. Target Grading
    im_grading_target = ax_target_grading.imshow(sample_grading_target, cmap='plasma')
    ax_target_grading.set_title('Target Grading', fontweight='bold')
    ax_target_grading.axis('off')
    plt.colorbar(im_grading_target, ax=ax_target_grading, fraction=0.046, pad=0.04)

    # 6. Predicted Grading
    im_grading_pred = ax_pred_grading.imshow(grading_prediction, cmap='plasma')
    ax_pred_grading.set_title('Predicted Grading', fontweight='bold')
    ax_pred_grading.axis('off')
    plt.colorbar(im_grading_pred, ax=ax_pred_grading, fraction=0.046, pad=0.04)

    # 7. Depth Difference
    depth_diff = np.abs(depth_prediction - sample_depth_target)
    im_depth_diff = ax_depth_diff.imshow(depth_diff, cmap='Reds')
    ax_depth_diff.set_title('Depth Error', fontweight='bold')
    ax_depth_diff.axis('off')
    plt.colorbar(im_depth_diff, ax=ax_depth_diff, fraction=0.046, pad=0.04)

    # 8. Grading Difference
    grading_diff = np.abs(grading_prediction - sample_grading_target)
    im_grading_diff = ax_grading_diff.imshow(grading_diff, cmap='Reds')
    ax_grading_diff.set_title('Grading Error', fontweight='bold')
    ax_grading_diff.axis('off')
    plt.colorbar(im_grading_diff, ax=ax_grading_diff, fraction=0.046, pad=0.04)

    # 9. Combined Prediction Visualization
    # Normalize and combine depth and grading for visualization
    depth_norm = (depth_prediction - depth_prediction.min()) / (depth_prediction.max() - depth_prediction.min())
    grading_norm = (grading_prediction - grading_prediction.min()) / (grading_prediction.max() - grading_prediction.min())
    combined = np.stack([depth_norm, grading_norm, np.zeros_like(depth_norm)], axis=-1)
    ax_combined.imshow(combined)
    ax_combined.set_title('Combined Output\n(R=Depth, G=Grading)', fontweight='bold')
    ax_combined.axis('off')

    # 10. Metrics Summary
    depth_mse = np.mean((depth_prediction - sample_depth_target) ** 2)
    grading_mse = np.mean((grading_prediction - sample_grading_target) ** 2)
    depth_mae = np.mean(np.abs(depth_prediction - sample_depth_target))
    grading_mae = np.mean(np.abs(grading_prediction - sample_grading_target))

    if loss_history:
        current_metrics = loss_history[-1]
        metrics_text = f"""INFERENCE RESULTS (Epoch {epoch})

Loss Values:
  Total: {current_metrics['total_loss']:.4f}
  Depth: {current_metrics['depth_loss']:.4f}
  Grading: {current_metrics['grading_loss']:.4f}

Prediction Errors:
  Depth MSE: {depth_mse:.6f}
  Depth MAE: {depth_mae:.6f}
  Grading MSE: {grading_mse:.6f}
  Grading MAE: {grading_mae:.6f}

Statistics:
  Depth Range: [{depth_prediction.min():.3f}, {depth_prediction.max():.3f}]
  Grading Range: [{grading_prediction.min():.2f}, {grading_prediction.max():.2f}]

Model Info:
  Layers: {n_layers}
  Feature Maps: {len(layer_features)}"""

        ax_metrics.text(0.05, 0.95, metrics_text, transform=ax_metrics.transAxes,
                       fontsize=9, verticalalignment='top', fontfamily='monospace',
                       bbox=dict(boxstyle="round,pad=0.5", facecolor="lightgray", alpha=0.8))
        ax_metrics.set_title('Metrics Summary', fontweight='bold')
        ax_metrics.axis('off')

    # 11. Statistics
    stats_text = f"""DETAILED STATISTICS

Target Stats:
  Depth Mean: {sample_depth_target.mean():.4f}
  Depth Std: {sample_depth_target.std():.4f}
  Grading Mean: {sample_grading_target.mean():.2f}
  Grading Std: {sample_grading_target.std():.2f}

Prediction Stats:
  Depth Mean: {depth_prediction.mean():.4f}
  Depth Std: {depth_prediction.std():.4f}
  Grading Mean: {grading_prediction.mean():.2f}
  Grading Std: {grading_prediction.std():.2f}

Correlation:
  Depth Corr: {np.corrcoef(depth_prediction.flatten(), sample_depth_target.flatten())[0,1]:.3f}
  Grading Corr: {np.corrcoef(grading_prediction.flatten(), sample_grading_target.flatten())[0,1]:.3f}"""

    ax_stats.text(0.05, 0.95, stats_text, transform=ax_stats.transAxes,
                 fontsize=9, verticalalignment='top', fontfamily='monospace',
                 bbox=dict(boxstyle="round,pad=0.5", facecolor="lightblue", alpha=0.8))
    ax_stats.set_title('Statistical Analysis', fontweight='bold')
    ax_stats.axis('off')

    # 12. Feature Maps from All Layers
    feature_map_idx = 0
    for layer_idx in range(n_layers):
        if layer_idx < len(layer_features) and layer_features[layer_idx] is not None:
            features = np.array(layer_features[layer_idx][0])  # Shape: (196, embed_dim)

            # Calculate spatial dimensions
            n_patches = features.shape[0]
            spatial_size = int(np.sqrt(n_patches))
            if spatial_size * spatial_size > n_patches:
                spatial_size -= 1
            adjusted_patches = spatial_size * spatial_size

            # Show 3 different views per layer
            views = ['magnitude', 'variance', 'channel_0']
            view_names = ['Magnitude', 'Variance', 'Ch-0']
            cmaps = ['viridis', 'plasma', 'RdBu_r']

            for view_idx, (view, view_name, cmap) in enumerate(zip(views, view_names, cmaps)):
                if feature_map_idx < len(feature_axes):
                    ax = feature_axes[feature_map_idx]

                    if view == 'magnitude':
                        feature_values = np.mean(np.abs(features), axis=1)[:adjusted_patches]
                    elif view == 'variance':
                        feature_values = np.var(features, axis=1)[:adjusted_patches]
                    else:  # channel_0
                        if features.shape[1] > 0:
                            feature_values = features[:adjusted_patches, 0]
                        else:
                            feature_values = np.zeros(adjusted_patches)

                    spatial_pattern = feature_values.reshape(spatial_size, spatial_size)

                    im = ax.imshow(spatial_pattern, cmap=cmap)
                    ax.set_title(f'L{layer_idx+1}-{view_name}', fontsize=10, fontweight='bold')
                    ax.axis('off')
                    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

                    feature_map_idx += 1

    # Fill remaining feature map axes
    for i in range(feature_map_idx, len(feature_axes)):
        feature_axes[i].axis('off')

    plt.suptitle(f'STRING CAYLEY ViT Comprehensive Analysis - Epoch {epoch}',
                 fontsize=16, fontweight='bold', y=0.98)

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
        print(f"📊 Comprehensive plot saved: {save_path}")

    plt.tight_layout()
    plt.show()

    return depth_prediction, grading_prediction, layer_features

def run_checkpoint_inference_demo(epoch: int = None):
    """
    🎯 Complete demo: Load checkpoint and create comprehensive visualization
    """
    print(f"\n{'='*80}")
    print("🎯 CHECKPOINT INFERENCE DEMONSTRATION")
    print(f"{'='*80}")

    # Load model from checkpoint
    model, config, epoch_loaded, history = load_model_from_checkpoint(epoch)
    if model is None:
        print("❌ Failed to load checkpoint")
        return None

    # Load some test data (use same data loading as training)
    try:
        print("📂 Loading test data...")
        images, depth_targets, grading_targets = load_dataset_for_training('/content', n_samples=10)
        print("✅ Real test data loaded")
    except:
        print("⚠️ Using synthetic test data...")
        images, depth_targets, grading_targets = load_synthetic_dataset(n_samples=10)

    # Convert to JAX arrays
    test_x = jnp.array(images)
    test_depth = jnp.array(depth_targets)
    test_grading = jnp.array(grading_targets)

    print(f"📊 Running inference on test data...")
    print(f"   🖼️ Test images: {test_x.shape}")
    print(f"   🏔️ Test depth targets: {test_depth.shape}")
    print(f"   📏 Test grading targets: {test_grading.shape}")

    # Create comprehensive visualization
    save_path = f'checkpoint_inference_epoch_{epoch_loaded}.png'
    depth_pred, grading_pred, features = comprehensive_inference_plot(
        model=model,
        test_data=(test_x, test_depth, test_grading),
        loss_history=history,
        epoch=epoch_loaded,
        save_path=save_path
    )

    print(f"✅ Inference demonstration completed!")
    print(f"📊 Comprehensive visualization generated")
    print(f"💾 Results saved to: {save_path}")

    return model, depth_pred, grading_pred, features

print("✅ Comprehensive visualization functions ready!")



📊 SECTION 6.5: COMPREHENSIVE VISUALIZATION
✅ Comprehensive visualization functions ready!


### 🎯 SECTION 7: MAIN EXECUTION

In [ ]:

if __name__ == "__main__":
    print("\n" + "="*80)
    print("🎯 MAIN EXECUTION - COMPLETE STRING CAYLEY VIT TRAINING")
    print("="*80)
    print("✅ Using NNX-compatible STRING CAYLEY attention mechanism")
    print("🔧 Fixed all compatibility issues between Linen and NNX")
    print("🎯 Dual spatial outputs: Depth maps + Grading maps")
    print("📂 Real data support with synthetic fallback")
    print("="*80)

    try:
        # Run training with complete NNX implementation
        model, loss_history, model_config = train_complete_string_vit(
            input_dir='/content',     # Adjust for your setup
            n_samples=200,            # Reasonable number for testing
            epochs=2000,                # Enough to see convergence
            batch_size=8,             # Manageable batch size
            learning_rate=0.001,
            n_layers=4,               # 4 transformer layers
            embed_dim=128,            # Larger embedding
            n_heads=8,                # More attention heads
            patch_size=16,
            save_checkpoints=True,    # Enable checkpointing
            checkpoint_interval=5     # Save every 5 epochs
        )

        print(f"\n🎊 SUCCESS! Complete STRING CAYLEY ViT training finished!")
    except Exception as e:
        print(f"\n❌ Training error: {e}")
        import traceback
        traceback.print_exc()

🔍 Starting layer processing with input shape: (1, 197, 128)
   Layer 1: Block output shape: (1, 197, 128)
   Layer 1: Patch features shape: (1, 196, 128)
   Layer 2: Block output shape: (1, 197, 128)
   Layer 2: Patch features shape: (1, 196, 128)
   Layer 3: Block output shape: (1, 197, 128)
   Layer 3: Patch features shape: (1, 196, 128)
   Layer 4: Block output shape: (1, 197, 128)
   Layer 4: Patch features shape: (1, 196, 128)
   Layer 5: Block output shape: (1, 197, 128)
   Layer 5: Patch features shape: (1, 196, 128)
   Layer 6: Block output shape: (1, 197, 128)
   Layer 6: Patch features shape: (1, 196, 128)
   Layer 7: Block output shape: (1, 197, 128)
   Layer 7: Patch features shape: (1, 196, 128)
   Layer 8: Block output shape: (1, 197, 128)
   Layer 8: Patch features shape: (1, 196, 128)
